# Setup

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
# define a surgery function if required
# don't run this unless you know what you're doing

def surgery(config):
    config["model_type"] = "LegacyRegression"

In [5]:
from accelerate import Accelerator
from utilities import try_loading_model

# set up the model
config = {"modelname": "smallandshort-21", "PATH": ".."}

# try loading model and config
model, config = try_loading_model(config, surgery_func=surgery)

accelerator = Accelerator()

model = accelerator.prepare(model)

/Users/midataur/Documents/github/braids/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded config from file, config may be different.
Did some surgery


W0201 11:56:00.786160 56421 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


In [27]:
model

LegacyRegression(
  (token_embedding_table): Embedding(9, 402)
  (position_embedding): Embedding(10, 402)
  (embed_hook): Identity()
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-5): 6 x Head(
            (key): Linear(in_features=402, out_features=67, bias=False)
            (query): Linear(in_features=402, out_features=67, bias=False)
            (value): Linear(in_features=402, out_features=67, bias=False)
            (dropout): Dropout(p=0, inplace=False)
            (attention_hook): Identity()
            (sanity_hook): Identity()
          )
        )
        (proj): Linear(in_features=402, out_features=402, bias=True)
        (dropout): Dropout(p=0, inplace=False)
      )
      (ffwd): FeedForward(
        (net): Sequential(
          (0): Linear(in_features=402, out_features=1608, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1608, out_features=402, bias=True)
          (3): Dropout(p=0, i

In [30]:
import numpy as np

lmao = model.lm_head.weight.cpu().detach().numpy()

px.imshow(lmao @ np.transpose(lmao))

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'coloraxis': 'coloraxis',
              'hovertemplate': 'x: %{x}<br>y: %{y}<br>color: %{z}<extra></extra>',
              'name': '0',
              'type': 'heatmap',
              'xaxis': 'x',
              'yaxis': 'y',
              'z': {'bdata': ('KCHwQXw6nL+6CSU/3NQ5v7ysHcC98r' ... 'K+vpxgx77i5pE9ZoLIwJj2AcG7eq5B'),
                    'dtype': 'f4',
                    'shape': '6, 6'}}],
    'layout': {'coloraxis': {'colorscale': [[0.0, '#0d0887'], [0.1111111111111111,
                                            '#46039f'], [0.2222222222222222,
                                            '#7201a8'], [0.3333333333333333,
                                            '#9c179e'], [0.4444444444444444,
                                            '#bd3786'], [0.5555555555555556,
                                            '#d8576b'], [0.6666666666666666,
                                            '#ed7953'], [0.7777777777777778,
                                            '#fb9f3a'], [0.8888888888888888,
                                            '#fdca26'], [1.0, '#f0f921']]},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'constrain': 'domain', 'domain': [0.0, 1.0], 'scaleanchor': 'y'},
               'yaxis': {'anchor': 'x', 'autorange': 'reversed', 'constrain': 'domain', 'domain': [0.0, 1.0]}}
})

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
import plotly.express as px

X = model.position_embedding.weight.cpu().detach().numpy()
pca = PCA(n_components=3)
pca.fit(X)

y = pca.transform(X)

px.scatter_3d(x=y[:,0],y=y[:,1],z=y[:,2], color="")

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'x=%{x}<br>y=%{y}<br>z=%{z}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'scene': 'scene',
              'showlegend': False,
              'type': 'scatter3d',
              'x': {'bdata': 'BGxXwTIB6sB5Xrc+F5VZQKVrjEANaxpAyhjLPxxxqr1NTd2/RRcnQQ==', 'dtype': 'f4'},
              'y': {'bdata': 'jl3hQLQH3r6A7pTAPMDUwEa5mcAPYUfAppGZv5S9Nb6A/s284D1gQQ==', 'dtype': 'f4'},
              'z': {'bdata': 'MpCWP6oG98CpZf7AihaHwKj2dT+0XJZACz2lQClXwUBrcK1AlYJkwA==', 'dtype': 'f4'}}],
    'layout': {'legend': {'tracegroupgap': 0},
               'margin': {'t': 60},
               'scene': {'domain': {'x': [0.0, 1.0], 'y': [0.0, 1.0]},
                         'xaxis': {'title': {'text': 'x'}},
                         'yaxis': {'title': {'text': 'y'}},
                         'zaxis': {'title': {'text': 'z'}}},
               'template': '...'}
})

In [41]:
y.shape

(10, 2)

# Embedding pictures

In [26]:
import plotly.express as px
import numpy as np
import torch

embedding = model.position_embedding.weight.cpu().detach().numpy()

similarity = []

for x in embedding:
    row = []
    for y in embedding:
        row.append(np.dot(x, y)/(np.linalg.norm(x)*np.linalg.norm(y)))
    similarity.append(row)

similarity = np.array(similarity)

figure = px.imshow(
    similarity,
    title=f"{config['modelname']} state embedding similarity matrix"
)

hide_ui = True

if hide_ui:
    figure.update_layout(coloraxis_showscale=True)
    figure.update_xaxes(showticklabels=False)
    figure.update_yaxes(showticklabels=False)
    figure.update_layout(
        title="",
        autosize=False,
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=0,
            pad=0
        )
    )

figure

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'coloraxis': 'coloraxis',
              'hovertemplate': 'x: %{x}<br>y: %{y}<br>color: %{z}<extra></extra>',
              'name': '0',
              'type': 'heatmap',
              'xaxis': 'x',
              'yaxis': 'y',
              'z': {'bdata': ('AQCAP9lEUT5fit68ksHuvT02tL0qhv' ... 'yj4SG9AUvkOhFfr7z5rbi7AACAPw=='),
                    'dtype': 'f4',
                    'shape': '10, 10'}}],
    'layout': {'autosize': False,
               'coloraxis': {'colorscale': [[0.0, '#0d0887'], [0.1111111111111111,
                                            '#46039f'], [0.2222222222222222,
                                            '#7201a8'], [0.3333333333333333,
                                            '#9c179e'], [0.4444444444444444,
                                            '#bd3786'], [0.5555555555555556,
                                            '#d8576b'], [0.6666666666666666,
                                            '#ed7953'], [0.7777777777777778,
                                            '#fb9f3a'], [0.8888888888888888,
                                            '#fdca26'], [1.0, '#f0f921']],
                             'showscale': True},
               'margin': {'b': 0, 'l': 0, 'pad': 0, 'r': 0, 't': 0},
               'template': '...',
               'title': {'text': ''},
               'xaxis': {'anchor': 'y',
                         'constrain': 'domain',
                         'domain': [0.0, 1.0],
                         'scaleanchor': 'y',
                         'showticklabels': False},
               'yaxis': {'anchor': 'x',
                         'autorange': 'reversed',
                         'constrain': 'domain',
                         'domain': [0.0, 1.0],
                         'showticklabels': False}}
})

In [7]:
config

{'modelname': 'smallandshort-21',
 'model_type': 'LegacyRegression',
 'dataset': 'smallandshort',
 'dataset_type': 'basicregression',
 'random_seed': 42,
 'braid_count': 4,
 'max_word_length': 10,
 'n_embed': 402,
 'n_heads': 6,
 'dropout': 0,
 'n_blocks': 8,
 'learning_rate': 0.00030000000000000003,
 'batchsize': 512,
 'weight_decay': 0.001,
 'lr_factor': 0.1,
 'lr_patience': 10,
 'threshold': 0.01,
 'n_workers': 0,
 'PATH': '..',
 'dataset_size': 4000000}

## True scratchpad

In [11]:
import torch

a = torch.tensor([1,2,3])

torch.exp(torch.abs(a))*a

tensor([ 2.7183, 14.7781, 60.2566])